In [ ]:
from keras import models, layers, activations, optimizers, losses, metrics
import os
import shutil

In [ ]:
#Download the cats_vs_dogs dataset from Kaggle
current_dir = '/home/hamza/Downloads/train'
base_dir = '/home/hamza/DL/dataset_cats_vs_dogs'

In [ ]:
os.mkdir(base_dir)

In [ ]:
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')
test_dir = os.path.join(base_dir,'test')

train_cats_dir = os.path.join(train_dir,'cats')
validation_cats_dir = os.path.join(validation_dir,'cats')
test_cats_dir = os.path.join(test_dir,'cats')


train_dogs_dir = os.path.join(train_dir,'dogs')
validation_dogs_dir = os.path.join(validation_dir,'dogs')
test_dogs_dir = os.path.join(test_dir,'dogs')

os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)

os.mkdir(train_cats_dir)
os.mkdir(validation_cats_dir)
os.mkdir(test_cats_dir)

os.mkdir(train_dogs_dir)
os.mkdir(validation_dogs_dir)
os.mkdir(test_dogs_dir)



In [ ]:
os.listdir(base_dir + '/train')

In [ ]:
train_cats = ['cat.{}.jpg'.format(i) for i in range(0,1000)]
train_dogs = ['dog.{}.jpg'.format(i) for i in range(0,1000)]

val_cats = ['cat.{}.jpg'.format(i) for i in range(1000,1500)]
val_dogs = ['dog.{}.jpg'.format(i) for i in range(1000,1500)]

test_cats = ['cat.{}.jpg'.format(i) for i in range(1500,2000)]
test_dogs = ['dog.{}.jpg'.format(i) for i in range(1500,2000)]


print(len(train_cats))
print(len(train_dogs))

print(len(val_cats))
print(len(val_dogs))

print(len(test_cats))
print(len(test_dogs))

In [ ]:
# for cat in train_cats:
#     src = os.path.join(current_dir, cat)
#     dst = os.path.join(train_cats_dir, cat)
#     shutil.copyfile(src, dst)
    
# for dog in train_dogs:
#     src = os.path.join(current_dir, dog)
#     dst = os.path.join(train_dogs_dir, dog)
#     shutil.copyfile(src, dst)
    
# for cat in val_cats:
#     src = os.path.join(current_dir, cat)
#     dst = os.path.join(validation_cats_dir, cat)
#     shutil.copyfile(src, dst)
    
# for dog in val_dogs:
#     src = os.path.join(current_dir, dog)
#     dst = os.path.join(validation_dogs_dir, dog)
#     shutil.copyfile(src, dst)
    
# for cat in test_cats:
#     src = os.path.join(current_dir, cat)
#     dst = os.path.join(test_cats_dir, cat)
#     shutil.copyfile(src, dst)
    
# for dog in test_dogs:
#     src = os.path.join(current_dir, dog)
#     dst = os.path.join(test_dogs_dir, dog)
#     shutil.copyfile(src, dst)
    

In [ ]:
def build_model():

    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(512, activation='relu'))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255) 
test_datagen =  ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')


In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50, verbose=0)

In [ ]:
model.save('cats_and_dogs_small_1.h5')


In [113]:
import matplotlib.pyplot as plt

%matplotlib

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Using matplotlib backend: TkAgg


In [104]:
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [107]:
# This is module with image preprocessing utilities
from keras.preprocessing import image

fnames = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]

# We pick one image to "augment"
img_path = fnames[3]

# Read the image and resize it
img = image.load_img(img_path, target_size=(150, 150))

# Convert it to a Numpy array with shape (150, 150, 3)
x = image.img_to_array(img)

# Reshape it to (1, 150, 150, 3)
x = x.reshape((1,) + x.shape)

# The .flow() command below generates batches of randomly transformed images.
# It will loop indefinitely, so we need to `break` the loop at some point!
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 1 == 0:
        break

plt.show()

In [108]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [111]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=50)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
100/100 [==============================] - 38s 384ms/step - loss: 0.6065 - acc: 0.6688 - val_loss: 0.5726 - val_acc: 0.6992
Epoch 2/10
100/100 [==============================] - 35s 352ms/step - loss: 0.6051 - acc: 0.6741 - val_loss: 0.5435 - val_acc: 0.7272
Epoch 3/10
100/100 [==============================] - 35s 346ms/step - loss: 0.5882 - acc: 0.6853 - val_loss: 0.5554 - val_acc: 0.7075
Epoch 4/10
100/100 [==============================] - 34s 341ms/step - loss: 0.5901 - acc: 0.6903 - val_loss: 0.5417 - val_acc: 0.7373
Epoch 5/10
100/100 [==============================] - 35s 346ms/step - loss: 0.5818 - acc: 0.6912 - val_loss: 0.5401 - val_acc: 0.7291
Epoch 6/10
100/100 [==============================] - 37s 371ms/step - loss: 0.5720 - acc: 0.6960 - val_loss: 0.5346 - val_acc: 0.7265
Epoch 7/10
100/100 [==============================] - 35s 348ms/step - loss: 0.5602 - acc: 0.6956 - val_lo

In [ ]:
model.save('cats_and_dogs_small_2.h5')

In [112]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()